
#### Read Immigration Dataset in SAS format AND Write as parquet format on S3.

In [1]:
import pandas as pd
import os
import configparser
from pyspark.sql import SparkSession
from datetime import datetime, timedelta, date
from pyspark.sql import types as t

In [2]:
# AWS credentials from configure file
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']     = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .getOrCreate()

print(spark)

In [41]:
months = ['apr','aug','dec','feb','jan','jul','jun','mar','may','nov','oct','sep']
i94ParquetPath = "Output_Data/i94.parquet" #"s3a://udacity-project-immigration2016-sas-files/"

In [42]:
for m in months:
    inputf = "{}{}{}".format("../../data/18-83510-I94-Data-2016/i94_",m,"16_sub.sas7bdat")
    print(inputf)
    df_spark_i94 = spark.read.format('com.github.saurfang.sas.spark').load(inputf)
    df_spark_i94.write.mode('append').partitionBy("i94mon").parquet(i94ParquetPath)

../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat


In [4]:
i94_table = spark.read.parquet('Output_Data/i94.parquet/i94mon=12.0/part-00000-9b8414f0-7ef9-4394-a1e2-9792dced91eb.c000.snappy.parquet')

In [5]:
i94_table.show()

+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|cicid| i94yr|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
| 46.0|2016.0| 129.0| 129.0|    HOU|20789.0|    1.0|     TX|20802.0|  46.0|    1.0|  1.0|20161201|     MDD| null|      H|      O|   null|      M| 1970.0|05262018|     M|  null|     RS| 9.755413803E10| 7715|      E2|
| 56.0|2016.0| 245.0| 245.0|    NEW|20789.0|    1.0|     OH|20835.0|  28.0|    3.0|  1.0|20161201|     BEJ| null|      U|      O|   null